#### Imports

In [161]:
import os
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

#### Get data from .txt files

In [162]:
directory_path = '../data/jobs_html/'
files = os.listdir(directory_path)
count = 0
for i in files:
    count += 1
count

507

In [163]:
items = []

remote_ratio = ['Presencial', 'Híbrido', 'En remoto']
employment_type = ['Jornada Completa', 'Media jornada', 'Prácticas']
experience_level = ['Sin experiencia', 'Intermedio', 'Algo de responsabilidad', 'Prácticas', 'Director', 'Ejecutivo']

for file_name in files:
    file_path = os.path.join(directory_path, file_name)
    row = []
    job_id = file_name.replace('.txt', '')
    row.append(job_id)
        
    if os.path.isfile(file_path):
        with open(file_path, 'r') as file:
            soup = BeautifulSoup(file.read(), 'html.parser')
            container = soup.find_all('div', {'class': 'jobs-unified-top-card'})[0]
            title = container.find_all('h1', {'class': 'job-details-jobs-unified-top-card__job-title'})[0].getText().strip()
            row.append(title)
            company_name = container.find_all('a', {'class': 'app-aware-link'})[1].getText().strip()
            row.append(company_name)
            state = container.find(string = lambda text: text and '· ' in text)
            row.append(state)
            line = container.find_all('li', {'class': 'job-details-jobs-unified-top-card__job-insight'})[0].getText().strip()
            line = line.split('\n')
            line = [i for i in line if i != '']
            line = [i for i in line if i != ' ']
            # print(line)
            if len(line) == 4:
                salary_range = line[0]
                remote = line[1]
                employment = line[2]
                experience = line[3]
                company_size = container.find_all('li', {'class': 'job-details-jobs-unified-top-card__job-insight'})[1].find('span').getText().strip()
            elif len(line) == 3:
                # print(line)
                if line[0] in remote_ratio and line[1] in employment_type and line[2] in experience_level:
                    salary_range = np.nan
                    remote = line[0]
                    employment = line[1]
                    experience = line[2]
                if '$' in line[0] and line[1] in employment_type and line[2] in experience_level:
                    salary_range = line[0]
                    remote = np.nan
                    employment = line[1]
                    experience = line[2]
                if '$' in line[0] and line[1] in remote_ratio and line[2] in employment_type:
                    salary_range = line[0]
                    remote = line[1]
                    employment = line[2]
                    experience = np.nan
                company_size = container.find_all('li', {'class': 'job-details-jobs-unified-top-card__job-insight'})[1].find('span').getText().strip()
            elif len(line) == 2:
                # print(line)
                if '$' in line[0] and line[1] in employment_type:
                    salary_range = line[0]
                    remote = np.nan
                    employment = line[1]
                    experience = np.nan
                if line[0] in remote_ratio and line[1] in employment_type:
                    salary_range = np.nan
                    remote = line[0]
                    employment = line[1]
                    experience = np.nan
                if line[0] in employment_type and line[1] in experience_level:
                    salary_range = np.nan
                    remote = np.nan
                    employment = line[0]
                    experience = line[1]

                company_size = container.find_all('li', {'class': 'job-details-jobs-unified-top-card__job-insight'})[1].find('span').getText().strip()
            row.append(salary_range)
            row.append(remote)
            row.append(employment)
            row.append(experience)
            row.append(company_size)
            items.append(row)

columns = ['job_id', 'job_title', 'company_name', 'company_state', 'salary_range', 'remote_ratio', 'employment_type', 'experience_level', 'company_size']

linkedin = pd.DataFrame(items, columns=columns)
linkedin.sample()

,job_id,job_title,company_name,company_state,salary_range,remote_ratio,employment_type,experience_level,company_size
479,3776532288,Data Analyst,Stirista,"· San Antonio, TX",144.000 $/año - 270.250 $/año,En remoto,Jornada completa,Intermedio,De 51 a 200 empleados · Servicios de publicidad


In [164]:
linkedin[linkedin['experience_level'] == 'Jornada completa']

,job_id,job_title,company_name,company_state,salary_range,remote_ratio,employment_type,experience_level,company_size


In [165]:
linkedin.to_csv('../data/linkedin.csv', index=False)